# Activity 1: Undo errors to a point in time

#### <i>The Azure SQL Workshop - Module 5</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In all organizations, big or small, accidents can happen. That's why you always have to have a plan for how you will restore to where you need to be. In SQL Server, ideally, you want choose to [restore to a point in time](https://docs.microsoft.com/en-us/sql/relational-databases/backup-restore/restore-a-sql-server-database-to-a-point-in-time-full-recovery-model?view=sql-server-ver15), but you can only do that if you are running in full recovery model. Under the bulk-logged recovery model, it's more likely that you'll have to recover the database to the end of the transaction log backup.  

One of the benefits of Azure SQL is that Azure can take care of all of this for you. Since Azure SQL manages your backups and runs in full recovery model, it can restore you to any point in time (you can even [restore a deleted database](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-recovery-using-backups#deleted-database-restore)). In this activity, you'll see how a common error can be recovered using point in time restore (PITR). This is easy to do in the portal or programmatically, but in this activity you'll see how to do it with the Azure CLI.  


**Set up**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the readme.md file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **PowerShell**. This determines what language the code blocks in the file are. In this case, that language is PowerShell.  
2. Confirming the Kernel is **PowerShell**, for "Attach to", it should read **localhost**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it. You should run the cells one at a time.    
 


### Process for PITR

Before you go any further, it's important to understand the recommended process for doing point in time restore (PITR):  

0. A table or database is deleted on accident.
1. Determine the time that you need to go back to. This should be **before** the error or mistake took place.  
1. Complete PITR via PowerShell or the Azure portal to go back to this time. This deploys a new database and restores a copy of your database, e.g. **AdventureWorks0406-copy**.  
1. Confirm the new database (e.g. **AdventureWorks0406-copy**) is in the correct state (before the accident occurred).  
1. Rename the original database, e.g. **AdventureWorks0406** to **AdventureWorks0406-old**.    
1. Rename the new database to the original database name, e.g. **AdventureWorks0406-copy** to **AdventureWorks0406**.  
1. Delete the original database, e.g. **AdventureWorks0406-old**.  

In this notebook, you'll follow the steps that go along with the process above.

**Step 0 - Delete a table on *accident***  

First, let's confirm that the table we'll *accidentally* delete does exist and have data in it. Let's take a look at some of the values in `SalesLT.OrderDetail`.  

**Navigate to SSMS** and **update your connection**. You'll want to make sure that the connection you use is connecting to the logical server, but not a specific database (e.g. set to `<default>` in screenshot below). You should also confirm that **Additional Connection Parameters** is contains no text.   

![](./graphics/default.png)

Next, right-click on your AdventureWorks database and create a new query. Run the following query and review the results.  

```sql
SELECT TOP 10 * from SalesLT.SalesOrderDetail
```

![](./graphics/salesdetailssms.png)  


For whatever reason, let's create a scenario where someone accidentally deletes that table. Today, you will be that someone.  

**Using the same query window**, run the following query.

```sql
DROP TABLE SalesLT.SalesOrderDetail
```




**Set up Azure CLI**

Finally, before you get into the steps, fill in your **Subscription ID** and **Workshop ID** (unique ID you've been using for other resources, e.g. `0406` or `168563`) information so the rest goes smoothly.  

You can then log in to the Azure portal and set the subscription.  

Run the cells before moving to **Step 1**.  

> Note: if you want to use this notebook for other databases and subscriptions in the future, you just need to replace the Subscription ID, Database Name, Logical server, and Resource group in the cell below to match your environment.  


In [ ]:
$subscription_id = "<SubscriptionID>"
$unique_id = "<WorkshopID>"
$database_name = "AdventureWorks$($unique_id)"
$database_name_copy = "$($database_name)-copy"
$database_name_old = "$($database_name)-old"
$logical_server = "aw-server$($unique_id)"
$resource_group = "azuresqlworkshop$($unique_id)"
Write-Output "Variables Received"

In [ ]:
# Log in to the Azure portal with your workshop credentials
# You may get an error initially, run again, and you should get a pop-up that directs you through authenticating
az login

In [ ]:
# Specify your subscription for the workshop
az account set --subscription $subscription_id

# Confirm you're connected to the correct subscription
az account show

In [ ]:
# Specify your default subscription, resource group, and Azure SQL Database logical server
az configure --defaults group=$resource_group sql-server=$logical_server

# Confirm the defaults have been set
az configure --list-defaults

**Step 1 - Determine the time you need to go back to using Audit logs**   

The first step is to figure out when you should restore the database. In order to complete step 1, you need to know when the last "good" transaction occurred, before the "bad" one, so you can restore to before the "bad" transaction but after the last "good" one.  

You may know ways to determine this time, but a new way may be to use the Audit logs in the Azure portal. Navigate to your Azure SQL Database in the Azure portal, e.g. **AdventureWorks0406**. In the left-hand menu, under Security, select **Auditing** and then select **View audit logs**.  

![](./graphics/viewauditlogs.png)  

Then, click on **Log Analytics**. If you see a *Get Started* screen, select **OK**. This then takes you to a query editor but it is not T-SQL. This view allows you to query logs using Kusto query language or KQL, which is meant to be easy to use and understand by SQL professionals. For the KQL documentation, [refer here](https://docs.microsoft.com/en-us/azure/kusto/query/).  

![](./graphics/loganalytics.png)  

Next, copy the below KQL query and paste it into the query editor in the Log Analytics view in the Azure portal (replace the existing query). **Update AdventureWorks0406 to match your database name**.

```kql
search database_name_s == "AdventureWorks0406"
| where Category == 'SQLSecurityAuditEvents' and statement_s like 'DROP'
| project format_datetime(event_time_t, 'yyyy-MM-dd hh:mm:ss.fff'), ResourceGroup, server_instance_name_s, database_name_s,  statement_s, succeeded_s,client_ip_s, server_principal_name_s, application_name_s
| sort by event_time_t desc
```

Once you've updated the above query in the Log Analytics query editor, select **Run** and review the results. It should be similar to below, but with a different date/time.   

> Note: If you see other `DROP`s, you should select the one related to the table you just dropped.  

![](./graphics/laresults.png)  

> NOTE: This can take a few minutes to show up here. If you are waiting for longer than 3-5 minutes, if you left the query window open in SSMS when you dropped the table, you can grab the drop time from there. But know, that in a real world situation, it is highly unlikely you will be able to get to that window with the completion time, so using auditing can help greatly.  

You'll notice, in this example, the date/time is `2020-02-10 09:28:54.509`. The required format is slightly different. Update it using this example as a reference and to the definition of `$before_error_time`.  
* Log Analytics format: `2020-02-10 09:28:54.509`
* Required format: `2020-02-10T21:28:54.509`  





In [ ]:
$before_error_time ="2020-03-18T17:43:40.438"
Write-Output "Variable Received"

**Step 2 - Complete PITR using the Azure CLI**  

In the next step you'll use `az cli db restore` to restore to before the table was deleted.  



In [ ]:
# Restore the database to the time before the database was deleted
az sql db restore --dest-name $database_name_copy --name $database_name --time $before_error_time --verbose

This above command will take about 3-5 minutes. This is because, in the background, Azure is deploying a new Azure SQL Database in your Azure SQL Database logical server that has all the same configuration options as the original. After it's deployed, it will then restore the database into that new Azure SQL Database.  

You can check the status by refreshing your view of databases in **SSMS** by right-clicking on **Databases** and selecting **Refresh**. Once the database has been deployed, you will see the restore is now in progress.  

![](./graphics/dbrestoring.png)  

Once you see this, it should only be 1-2 minutes more. You will know it is done, because the "stop" button in the code cell above will stop spinning and go back to the standard "play" button. Also, you will no longer see "(Restoring...)" next to the copy database when you initiate a refresh.  

If you notice it is taking longer than above stated times, it could be due to your environment. There are a limited number of restore requests that can be processed/submitted at once for a single subscription. If you want to learn more about the limits and related details for PITR while you wait, you can [read more to learn the details related to recovering an Azure SQL database by using automated database backups](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-recovery-using-backups). 

**Step 3 - Confirm the new database is in the correct state (before the accident occurred)**

In order to do this, refresh your connection to the Azure SQL Database logical server in SSMS (right-click on the logical server and select **Refresh**).  

Then, right-click on your new database, e.g. **AdventureWorks0406-copy** and select **New Query**.  

![](./graphics/newnewquery.png)  

Use the following query to confirm the table exists.  

```sql
SELECT TOP 10 * from SalesLT.SalesOrderDetail
```
You should get something similar to the following screenshot, which confirms your database has been restored to where you want it to be.    

![](./graphics/salesdetailssms.png)  




**Step 4 - Rename the original database**  

This step involves renaming the original database to something similar to **AdventureWorks0406-old** so you can later rename the new database to the original database name. As long as your applications use retry logic, this will make it so no connection strings need to be changed.  

Now, you're very familiar with how to rename databases in SSMS, but here you will see how it can be easily done using the Azure CLI.  

> Note: If at any point your database appears unavailable (e.g. you can't connect to the databases in SSMS if you refresh the connection), it could be due to updates happening to the DNS table. So while the database isn't physically unavailable, it is unresolvable. If you wait a minute or so, you should be able to resume normal activities.  



In [ ]:
az sql db rename --name $database_name --new-name $database_name_old

**Step 5 - Rename the new database to the original database name**  

Now that the original database name is no longer taken, you can rename the copy database to that of the original, again using the Azure CLI.  


In [ ]:
az sql db rename --name $database_name_copy --new-name $database_name

**Step 6 - Delete the original database**  

Finally, you have no need for the old database, so you can delete it with `az sql db delete`.  


In [ ]:
az sql db delete --name $database_name_old --yes
Write-Host "Database deleted"

And you can confirm it no longer exists with the following command.

In [ ]:
az sql db list -o table

You've now seen how you can leverage point in time restore (PITR) in Azure SQL Database. PITR is also available in Azure SQL Managed Instance, **for databases not the whole instance**. You can use almost the same commands except with `az sql midb` as opposed to `az sql db`. For more information, see the [documentation](https://docs.microsoft.com/en-us/cli/azure/sql/midb?view=azure-cli-latest#az-sql-midb-restore).  


